get NPP from MYD MOD17AHGF

year: 2001 - 2020

Point: Japanese 4rd time Mesh 0.5 km centroid

## NNP Extraction
Net Primary Productivity

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import gdal
import os
import glob
import rasterio

In [6]:
#year = 2015
aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
os.mkdir(aimFolder + "\\temp")

In [9]:
def mergeAndResampleNppRaster(year, aimFolder):
    #"""
    #Merge MOD and MYD,
    #Resample the raster with 0.004 degrees
    #"""
    rasterFile = glob.glob("F:/17_Article/01_Data/02_NetPrimaryProductivity/Net_PP_Yearly_500m_v6/Npp/*"+str(year) +"_001.tif")

    if year > 2001:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)
        tifflayer_1 = gdal.Open(rasterFile[1], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0

        tifflayer_1_array = tifflayer_1.ReadAsArray()
        tifflayer_1_array = np.array(tifflayer_1_array)
        tifflayer_1_array = tifflayer_1_array.astype(float)
        tifflayer_1_array[tifflayer_1_array == 65535] = np.nan
        tifflayer_1_array[tifflayer_1_array > 32760] = 0

        tifflayer_double_array = np.array([tifflayer_0_array, tifflayer_1_array])
        tifflayer_mean_array = np.nanmean(tifflayer_double_array, axis = 0)
    else:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0
        tifflayer_mean_array = tifflayer_0_array 


    ncol = tifflayer_mean_array.shape[1]
    nrow = tifflayer_mean_array.shape[0]

    temp_file = aimFolder + "/temp/temp_file_" + str(year) + ".tif"

    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(temp_file, ncol, nrow, nband, gdal.GDT_Float32)
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(tifflayer_mean_array)
    dst_dataset = None

    original_temp_raster = gdal.Open(temp_file, gdal.GA_ReadOnly)
    raster_rprj = gdal.Warp(aimFolder + "/temp/temp_file_" + str(year) + "0008.tif", original_temp_raster, dstSRS = "EPSG:4326", 
                            xRes = 0.004, yRes = 0.004, resampleAlg = "average")
    original_temp_raster = None
    raster_rprj = None


In [7]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

Parallel(n_jobs=10)(delayed(mergeAndResampleNppRaster)(int(year), aimFolder) for year in np.linspace(2002, 2020, 20))

IndexError: list index out of range

In [8]:
np.linspace(2001, 2020, 20)

array([2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009.,
       2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017., 2018.,
       2019., 2020.])

In [15]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")

In [73]:
#os.remove(temp_file)
#os.remove(aimFolder + "/temp/temp_file_0008.tif")

#os.rmdir(aimFolder + "\\temp")

In [75]:
def nppExtraction(year, aimFolder, coords_extration):
    # this is to extract npp of each eyar of each mesh point
    rasterFile = rasterio.open(aimFolder + "/temp/temp_file_" + str(year) + "0004.tif")
    rasterArray = rasterFile.read(1)

    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration['NPP_'+str(year)] = valueArray

    rasterFile = None
    return coords_extration

In [76]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
nppResult = Parallel(n_jobs=4)(delayed(nppExtraction)(int(year), aimFolder, coords_extration) for year in np.linspace(2001, 2020, 20))

## NTL extraction
Night Time Light

In [3]:
import os
import zipfile
import glob
import rasterio
from osgeo import gdal
import geopandas as gpd
import numpy as np

In [4]:
aimFolder = "F:\\17_Article\\01_Data\\03_NTL_VIIRS"
os.mkdir(aimFolder + "\\temp")

In [12]:
def ntlExtraction(year, aimFolder, coords_extration):
    #This funciton is try to extract ntl
    #filename = aimFolder + "\\" + str(year) + ".zip"
    #with zipfile.ZipFile(filename, "r") as zip_ref:
    #    zip_ref.extractall(aimFolder + "\\temp")
    aimRaster = glob.glob(aimFolder + "\\temp\\*" + str(year) + ".tif")
    aimRaster = aimRaster[0]
    rasterFile = rasterio.open(aimRaster)
    rasterArray = rasterFile.read(1)
    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
    
    valueArray = np.array(valueArray)
    coords_extration['NTL_'+str(year)] = valueArray
    return coords_extration

In [16]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

ntlResult = Parallel(n_jobs=2)(delayed(ntlExtraction)(int(year), aimFolder, coords_extration) for year in np.linspace(2002, 2020, 19))

In [36]:
ntlResult_2001 = ntlExtraction(int(2001), aimFolder, coords_extration)

In [37]:
ntlResult_2002 = ntlResult[0]
ntlResult_2003 = ntlResult[1]
ntlResult_2004 = ntlResult[2]
ntlResult_2005 = ntlResult[3]
ntlResult_2006 = ntlResult[4]
ntlResult_2007 = ntlResult[5]
ntlResult_2008 = ntlResult[6]
ntlResult_2009 = ntlResult[7]
ntlResult_2010 = ntlResult[8]
ntlResult_2011 = ntlResult[9]
ntlResult_2012 = ntlResult[10]
ntlResult_2013 = ntlResult[11]
ntlResult_2014 = ntlResult[12]
ntlResult_2015 = ntlResult[13]
ntlResult_2016 = ntlResult[14]
ntlResult_2017 = ntlResult[15]
ntlResult_2018 = ntlResult[16]
ntlResult_2019 = ntlResult[17]
ntlResult_2020 = ntlResult[18]

In [38]:
ntlResult_2001['year'] = 2001
ntlResult_2002['year'] = 2002
ntlResult_2003['year'] = 2003
ntlResult_2004['year'] = 2004
ntlResult_2005['year'] = 2005
ntlResult_2006['year'] = 2006
ntlResult_2007['year'] = 2007
ntlResult_2008['year'] = 2008
ntlResult_2009['year'] = 2009
ntlResult_2010['year'] = 2010
ntlResult_2011['year'] = 2011
ntlResult_2012['year'] = 2012
ntlResult_2013['year'] = 2013
ntlResult_2014['year'] = 2014
ntlResult_2015['year'] = 2015
ntlResult_2016['year'] = 2016
ntlResult_2017['year'] = 2017
ntlResult_2018['year'] = 2018
ntlResult_2019['year'] = 2019
ntlResult_2020['year'] = 2020

In [40]:
ntlResult_2001 = pd.DataFrame(ntlResult_2001.drop(columns='geometry'))
ntlResult_2002 = pd.DataFrame(ntlResult_2002.drop(columns='geometry'))
ntlResult_2003 = pd.DataFrame(ntlResult_2003.drop(columns='geometry'))
ntlResult_2004 = pd.DataFrame(ntlResult_2004.drop(columns='geometry'))
ntlResult_2005 = pd.DataFrame(ntlResult_2005.drop(columns='geometry'))
ntlResult_2006 = pd.DataFrame(ntlResult_2006.drop(columns='geometry'))
ntlResult_2007 = pd.DataFrame(ntlResult_2007.drop(columns='geometry'))
ntlResult_2008 = pd.DataFrame(ntlResult_2008.drop(columns='geometry'))
ntlResult_2009 = pd.DataFrame(ntlResult_2009.drop(columns='geometry'))
ntlResult_2010 = pd.DataFrame(ntlResult_2010.drop(columns='geometry'))
ntlResult_2011 = pd.DataFrame(ntlResult_2011.drop(columns='geometry'))
ntlResult_2012 = pd.DataFrame(ntlResult_2012.drop(columns='geometry'))
ntlResult_2013 = pd.DataFrame(ntlResult_2013.drop(columns='geometry'))
ntlResult_2014 = pd.DataFrame(ntlResult_2014.drop(columns='geometry'))
ntlResult_2015 = pd.DataFrame(ntlResult_2015.drop(columns='geometry'))
ntlResult_2016 = pd.DataFrame(ntlResult_2016.drop(columns='geometry'))
ntlResult_2017 = pd.DataFrame(ntlResult_2017.drop(columns='geometry'))
ntlResult_2018 = pd.DataFrame(ntlResult_2018.drop(columns='geometry'))
ntlResult_2019 = pd.DataFrame(ntlResult_2019.drop(columns='geometry'))
ntlResult_2020 = pd.DataFrame(ntlResult_2020.drop(columns='geometry'))

In [41]:
ntlResult_2001.columns

Index(['G04c_001', 'NTL_2001', 'year'], dtype='object')

In [45]:
ntlResult_2001.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2002.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2003.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2004.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2005.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2006.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2007.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2008.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2009.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2010.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2011.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2012.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2013.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2014.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2015.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2016.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2017.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2018.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2019.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2020.columns = ['G04c_001', 'NTL', 'year']

In [57]:
ntlResult_2020.NTL[ntlResult_2020["NTL"]<0] = 0
ntlResult_2020["NTL"].mean()
ntlResult_2019.NTL[ntlResult_2019["NTL"]<0] = 0
ntlResult_2019["NTL"].mean()


C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\1752473119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ntlResult_2020.NTL[ntlResult_2020["NTL"]<0] = 0
C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\1752473119.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ntlResult_2019.NTL[ntlResult_2019["NTL"]<0] = 0


1.1114916

In [60]:
print(ntlResult_2001["NTL"].mean())
print(ntlResult_2002["NTL"].mean())
print(ntlResult_2003["NTL"].mean())
print(ntlResult_2004["NTL"].mean())
print(ntlResult_2005["NTL"].mean())
print(ntlResult_2006["NTL"].mean())
print(ntlResult_2007["NTL"].mean())
print(ntlResult_2008["NTL"].mean())
print(ntlResult_2009["NTL"].mean())
print(ntlResult_2010["NTL"].mean())
print(ntlResult_2011["NTL"].mean())
print(ntlResult_2012["NTL"].mean())
print(ntlResult_2013["NTL"].mean())
print(ntlResult_2014["NTL"].mean())
print(ntlResult_2015["NTL"].mean())
print(ntlResult_2016["NTL"].mean())
print(ntlResult_2017["NTL"].mean())
print(ntlResult_2018["NTL"].mean())
print(ntlResult_2019["NTL"].mean())
print(ntlResult_2020["NTL"].mean())

1.6480222
1.6131924
1.4684126
1.2629389
1.5879596
1.575272
1.6215699
1.4386377
1.3462512
1.0992323
1.3615805
1.3009588
1.061045
1.0638536
1.0337858
1.0159932
1.1423855
1.1576278
1.1114916
1.0489889


In [61]:
longNtlDF = pd.concat(
    [ntlResult_2001, ntlResult_2002, ntlResult_2003, ntlResult_2004, ntlResult_2005, ntlResult_2006, ntlResult_2007, ntlResult_2008,
    ntlResult_2009, ntlResult_2010, ntlResult_2011, ntlResult_2012, ntlResult_2013, ntlResult_2014, ntlResult_2015, ntlResult_2016,
    ntlResult_2017, ntlResult_2018, ntlResult_2019, ntlResult_2020]
)

In [64]:
del ntlResult_2001
del ntlResult_2002
del ntlResult_2003
del ntlResult_2004
del ntlResult_2005
del ntlResult_2006
del ntlResult_2007
del ntlResult_2008
del ntlResult_2009
del ntlResult_2010
del ntlResult_2011
del ntlResult_2012
del ntlResult_2013
del ntlResult_2014
del ntlResult_2015
del ntlResult_2016
del ntlResult_2017
del ntlResult_2018
del ntlResult_2019
del ntlResult_2020

import gc
gc.collect()

176

In [62]:
longNtlDF = longNtlDF.set_index(['G04c_001','year'])
longNtlDF.head()

,,NTL
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


In [63]:

longNtlDF.mean()

NTL    1.297961
dtype: float32

In [65]:
del ntlResult
gc.collect()

156

In [71]:
fileList = glob.glob(aimFolder + "\\temp\\*")
#fileList = fileList[1:]
for filename in fileList:
    os.remove(filename)

#os.rmdir(aimFolder + "\\temp")

PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2001.tif'

In [ ]:
os.rmdir(aimFolder + "\\temp")

['F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2001.tfw',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2001.tif',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2001.tif.aux.xml',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2001.tif.xml',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2002.tfw',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2002.tif',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2002.tif.aux.xml',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2002.tif.xml',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2003.tfw',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2003.tif',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2003.tif.aux.xml',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2003.tif.xml',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2004.tfw',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2004.tif',
 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\